In [54]:
import geopandas as gpd

from random import randint
from keplergl import KeplerGl
import csv
from tqdm import tqdm

In [2]:
images = images = gpd.read_file('Geo-JSON Files/image_info.geojson')

In [60]:
with open('Triplets.csv', mode='a+', newline='', encoding='utf-8') as csvfile:
	csv_writer = csv.writer(csvfile)
	
	if csvfile.tell() == 0:
		csv_writer.writerow(['Anchor ID', 'Positive ID', 'Negative ID'])
	
	for i in tqdm(range(1000)):
		random_image_number = randint(0, len(images) - 1)

		potential_anchor = images.iloc[random_image_number]
		potential_anchor_frame = gpd.GeoDataFrame(geometry=[potential_anchor['geometry']])
		buffer_distance = 0.0001 # 10m radius

		buffer_polygon = potential_anchor['geometry'].buffer(buffer_distance)
		buffer_polygon_frame = gpd.GeoDataFrame(geometry=[buffer_polygon])

		potential_positives = images[images['geometry'].within(buffer_polygon) & (images['img_id'] != images.iloc[random_image_number]['img_id']) & (abs(images['year'] - images.iloc[random_image_number]['year']) <= 3) & (images['part_of_img'] == images.iloc[random_image_number]['part_of_img'])]
		
		if len(potential_positives) < 1:
			continue
		
		random_positive_number = randint(0, len(potential_positives) - 1)
		potential_positive = potential_positives.iloc[random_positive_number]
		potential_positive_frame = gpd.GeoDataFrame(geometry=[potential_positive['geometry']])

		negative_buffer_inner = potential_anchor['geometry'].buffer(0.0005)
		negative_buffer_outer = potential_anchor['geometry'].buffer(0.0025)

		negative_buffer_inner_frame = gpd.GeoDataFrame(geometry=[negative_buffer_inner])
		negative_buffer_outer_frame = gpd.GeoDataFrame(geometry=[negative_buffer_outer])

		negative_buffer = negative_buffer_outer.difference(negative_buffer_inner)
		negative_buffer_frame = gpd.GeoDataFrame(geometry=[negative_buffer])

		# potential_negatives = images[images['geometry'].within(negative_buffer) & (images['year'] == potential_positives.iloc[random_positive_number]['year']) & (images['month'] == potential_positives.iloc[random_positive_number]['month'])]
		potential_negatives = images[images['geometry'].within(negative_buffer) & (images['year'] == potential_positive['year']) & (images['month'] == potential_positive['month'])]
		
		if len(potential_negatives) < 1:
			continue

		random_negative_number = randint(0, len(potential_negatives) - 1)
		potential_negative = potential_negatives.iloc[random_negative_number]
		potential_negative_frame = gpd.GeoDataFrame(geometry=[potential_negative['geometry']])


		triplet = (potential_anchor['img_id_com'], potential_positive['img_id_com'], potential_negative['img_id_com'])
		csv_writer.writerow(triplet)


100%|██████████| 1000/1000 [01:33<00:00, 10.66it/s]


In [15]:
random_image_number = randint(0, len(images))

potential_anchor = images.iloc[random_image_number]
potential_anchor = images.iloc[35531]['geometry']
potential_anchor_frame = gpd.GeoDataFrame({'geometry': [potential_anchor]})
print(potential_anchor)
buffer_distance = 0.001 # 100m radius
buffer_distance = 0.0001 # 10m radius

buffer_polygon = potential_anchor.buffer(buffer_distance)
buffer_polygon_frame = gpd.GeoDataFrame(geometry=[buffer_polygon])
print(buffer_polygon)

POINT (4.351028575396204 51.9902783318869)
POLYGON ((4.351128575396204 51.9902783318869, 4.351128093868871 51.99026853017286, 4.351126653924244 51.9902588228547, 4.351124269429777 51.990249303419176, 4.351120963349455 51.99024006354366, 4.3511167675226385 51.990231192213216, 4.351111722357434 51.9902227748636, 4.35110587644154 51.990214892558484, 4.351099286074323 51.99020762120878, 4.35109201472462 51.99020103084156, 4.351084132419506 51.99019518492567, 4.351075715069887 51.990190139760465, 4.35106684373944 51.990185943933646, 4.3510576038639295 51.99018263785332, 4.351048084428405 51.99018025335886, 4.351038377110237 51.99017881341423, 4.351028575396204 51.990178331886895, 4.351018773682171 51.99017881341423, 4.351009066364003 51.99018025335886, 4.350999546928478 51.99018263785332, 4.350990307052967 51.990185943933646, 4.350981435722521 51.990190139760465, 4.350973018372902 51.99019518492567, 4.350965136067788 51.99020103084156, 4.350957864718085 51.99020762120878, 4.350951274350868 

In [45]:
potential_positives = images[images['geometry'].within(buffer_polygon) & (images['img_id'] != images.iloc[35531]['img_id']) & (abs(images['year'] - images.iloc[35531]['year']) <= 5) & (images['part_of_img'] == images.iloc[35531]['part_of_img'])]
random_positive_number = randint(0, len(potential_positives))
potential_positive = potential_positives.iloc[random_positive_number]
potential_positive = potential_positives.iloc[2]['geometry']

In [46]:
negative_buffer_inner = potential_anchor.buffer(0.0005)
negative_buffer_outer = potential_anchor.buffer(0.0025)

negative_buffer_inner_frame = gpd.GeoDataFrame(geometry=[negative_buffer_inner])
negative_buffer_outer_frame = gpd.GeoDataFrame(geometry=[negative_buffer_outer])

negative_buffer = negative_buffer_outer.difference(negative_buffer_inner)
negative_buffer_frame = gpd.GeoDataFrame(geometry=[negative_buffer])

In [47]:
potential_negatives = images[images['geometry'].within(negative_buffer) & (images['year'] == potential_positives.iloc[2]['year']) & (images['month'] == potential_positives.iloc[2]['month'])]

In [48]:
map_1 = KeplerGl(height=600)
map_1.add_data(data=images, name='images')
map_1.add_data(data=potential_anchor_frame, name='potential anchor')
map_1.add_data(data=buffer_polygon_frame, name='buffer polygon')
map_1.add_data(data=potential_positives, name='potential positives')
map_1.add_data(data=negative_buffer_frame, name='negative buffer')
map_1.add_data(data=potential_negatives, name='potential negatives')
map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'images': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, …